# GETTING DATA FROM API #


This is the data wrangling exercise as part of the IBM Data Science course, however, I have done it differently to suit my personal style of data organisation and methodology. I do not like accessing urls multiple times, instead finding more convenience in initialising dictionaries and filling the relevant values in the datasets.

In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import numpy as np
import datetime as dt

In [271]:
pd.set_option('display.max_columns',None) #display all column names in the dataframe
pd.set_option('display.max_colwidth',None) #display all the data in the dataframe

In [309]:
spacex_url = "https://api.spacexdata.com/v4/launches/past"
spacex_req = requests.get(spacex_url) #using the API means we get a JSON file
spacex_df = pd.json_normalize(spacex_req.json()) #basically the read json is json_normalize

In [310]:
spX = ['rocket','payloads','launchpad','cores','flight_number','date_utc']
spacex_df = spacex_df[spX]

Filtering the data as per the requirements:
1. remove rows with multiple cores
2. for payloads and cores, extract the single value in the list and replace the feature
3. convert the date_utc to a datetime datatype and then extracting the date
4. keep all the dates before (2020,11,13) 

In [311]:
#Data preparation as per the requirements completed
spacex_df = spacex_df[spacex_df.cores.map(len)==1]
spacex_df['cores'] = spacex_df.cores.map(lambda x: x[0])
spacex_df = spacex_df[spacex_df.payloads.map(len)==1]
spacex_df['payloads'] = spacex_df.payloads.map(lambda x: x[0])

spacex_df['date'] = pd.to_datetime(spacex_df['date_utc']).dt.date
spacex_df = spacex_df[spacex_df.date <= dt.date(2020, 11, 13)]

Keeping only Falcon 9 rocket, using the text file to find the corresponding name and ID for Falcon 9

In [312]:
#to replace the 'rocket' column with the engine names and keep only Falcon9

spacex_df['rocket'] = spacex_df['rocket'].replace('5e9d0d95eda69973a809d1ec','Falcon 9')
spacex_df = spacex_df[spacex_df['rocket'] == 'Falcon 9'].reset_index().drop('index',axis=1)

In [313]:
#dropping date_utc
spacex_df = spacex_df.drop('date_utc',axis=1)

In [142]:
# Takes the dataset and uses the payloads column to call the API and append the data to the lists
def getPayloadData(data):
    for load in data['payloads']:
        response = requests.get("https://api.spacexdata.com/v4/payloads/"+load).json()
        PayloadMass.append(response['mass_kg'])
        Orbit.append(response['orbit'])

Getting the launch site details in a way I prefer, that actually matches the ids in the spacex_df with the corresponding location, and  you are only accessing the url once. I think this is much cleaner than the prescribed method.

In [314]:
launch_url = "https://api.spacexdata.com/v4/launchpads"
launch_response = requests.get(launch_url)

In [315]:
#launch_temp = pd.json_normalize(launch_response.json())
#launch_temp = launch_temp[['name','id','latitude','longitude']]
#temp = launch_temp.values
locations, ids = list(temp[:,0]), list(temp[:,1])
latitude, longitude = list(temp[:,2]), list(temp[:,3])

In [316]:
spacex_df['launchpad'] = spacex_df['launchpad'].replace(ids,locations)

In [317]:
spacex_df.head()

,rocket,payloads,launchpad,cores,flight_number,date
0,Falcon 9,5eb0e4b7b6c3bb0006eeb1e7,CCSFS SLC 40,"{'core': '5e9e289ef359185f2b3b2628', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",6,2010-06-04
1,Falcon 9,5eb0e4bab6c3bb0006eeb1ea,CCSFS SLC 40,"{'core': '5e9e289ef35918f39c3b262a', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",8,2012-05-22
2,Falcon 9,5eb0e4bbb6c3bb0006eeb1ed,CCSFS SLC 40,"{'core': '5e9e289ff3591884e03b262c', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",10,2013-03-01
3,Falcon 9,5eb0e4bbb6c3bb0006eeb1ee,VAFB SLC 4E,"{'core': '5e9e289ff359180ae23b262d', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': True, 'landing_success': False, 'landing_type': 'Ocean', 'landpad': None}",11,2013-09-29
4,Falcon 9,5eb0e4bbb6c3bb0006eeb1ef,CCSFS SLC 40,"{'core': '5e9e289ff35918862c3b262e', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",12,2013-12-03


In [318]:
spacex_df.cores[0].keys()

dict_keys(['core', 'flight', 'gridfins', 'legs', 'reused', 'landing_attempt', 'landing_success', 'landing_type', 'landpad'])

Making a separate for the cores to be added to spacex_df later

In [319]:
#developing the dataframe for cores
cores_df = pd.DataFrame(columns = list(spacex_df.cores[0].keys()))
for ix in range(spacex_df.shape[0]):
    cores_df = cores_df.append(spacex_df.cores.iloc[ix],ignore_index=True)
cores_df = cores_df.reset_index().drop('index',axis=1)

In [320]:
cores_df.shape[0] ==spacex_df.shape[0]

True

In [321]:
cores_df.head()

,core,flight,gridfins,legs,reused,landing_attempt,landing_success,landing_type,landpad
0,5e9e289ef359185f2b3b2628,1,False,False,False,False,None,None,None
1,5e9e289ef35918f39c3b262a,1,False,False,False,False,None,None,None
2,5e9e289ff3591884e03b262c,1,False,False,False,False,None,None,None
3,5e9e289ff359180ae23b262d,1,False,False,False,True,False,Ocean,None
4,5e9e289ff35918862c3b262e,1,False,False,False,False,None,None,None


From the head comparisons of spacex_df and cores_df, it is clear that the cores id matches the cores_df core column. Thus, we can move forward with concatenating the datasets.

In [322]:
cores_df = cores_df.drop('core',axis=1)
cores_df.head()

,flight,gridfins,legs,reused,landing_attempt,landing_success,landing_type,landpad
0,1,False,False,False,False,None,None,None
1,1,False,False,False,False,None,None,None
2,1,False,False,False,False,None,None,None
3,1,False,False,False,True,False,Ocean,None
4,1,False,False,False,False,None,None,None


*putting in the landpad sites*

In [323]:
temp_df = pd.concat([spacex_df,cores_df],axis=1)

In [324]:
spacex_df = temp_df.copy()

In [325]:
spacex_df.head()

,rocket,payloads,launchpad,cores,flight_number,date,flight,gridfins,legs,reused,landing_attempt,landing_success,landing_type,landpad
0,Falcon 9,5eb0e4b7b6c3bb0006eeb1e7,CCSFS SLC 40,"{'core': '5e9e289ef359185f2b3b2628', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",6,2010-06-04,1,False,False,False,False,None,None,None
1,Falcon 9,5eb0e4bab6c3bb0006eeb1ea,CCSFS SLC 40,"{'core': '5e9e289ef35918f39c3b262a', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",8,2012-05-22,1,False,False,False,False,None,None,None
2,Falcon 9,5eb0e4bbb6c3bb0006eeb1ed,CCSFS SLC 40,"{'core': '5e9e289ff3591884e03b262c', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",10,2013-03-01,1,False,False,False,False,None,None,None
3,Falcon 9,5eb0e4bbb6c3bb0006eeb1ee,VAFB SLC 4E,"{'core': '5e9e289ff359180ae23b262d', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': True, 'landing_success': False, 'landing_type': 'Ocean', 'landpad': None}",11,2013-09-29,1,False,False,False,True,False,Ocean,None
4,Falcon 9,5eb0e4bbb6c3bb0006eeb1ef,CCSFS SLC 40,"{'core': '5e9e289ff35918862c3b262e', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",12,2013-12-03,1,False,False,False,False,None,None,None


Working with payload data to get the columns for payload mass and orbit

In [328]:
payload_url = "https://api.spacexdata.com/v4/payloads"
payload_response = requests.get(payload_url)

In [329]:
payload = pd.json_normalize(payload_response.json())

In [426]:
payload_temp = pd.json_normalize(payload_response.json())
payload_temp = payload_temp[['mass_kg','orbit','id']]
launch_id, mass_kg, orbit = list(payload_temp.id), list(payload_temp.mass_kg), list(payload_temp.orbit)

In [427]:
temp_df = spacex_df.copy()

In [430]:
#create a dictionary with orbits against payload ids
orbit_dict = {}
for orbit in list(set(payload_temp.orbit.values)):
    orbit_dict[orbit] = list(payload_temp[payload_temp['orbit']==orbit]['id'].values)


In [439]:
#create a dictionary with mass against payload ids
mass_dict = {}
for mass in list(set(payload_temp.mass_kg.values)):
    mass_dict[mass] = list(payload_temp[payload_temp['mass_kg']==mass]['id'].values)

In [437]:
for key in orbit_dict.keys():
    orbit_list = orbit_dict[key]
    temp_df['orbit'] = temp_df['orbit'].replace(orbit_list,key)


In [440]:
for key in mass_dict.keys():
    mass_list = mass_dict[key]
    temp_df['mass_kg'] = temp_df['mass_kg'].replace(mass_list,key)

In [457]:
temp_df.head()

,rocket,payloads,launchpad,cores,flight_number,date,flight,gridfins,legs,reused,landing_attempt,landing_success,landing_type,landpad,mass_kg,orbit
0,Falcon 9,5eb0e4b7b6c3bb0006eeb1e7,CCSFS SLC 40,"{'core': '5e9e289ef359185f2b3b2628', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",6,2010-06-04,1,False,False,False,False,None,None,None,5eb0e4b7b6c3bb0006eeb1e7,LEO
1,Falcon 9,5eb0e4bab6c3bb0006eeb1ea,CCSFS SLC 40,"{'core': '5e9e289ef35918f39c3b262a', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",8,2012-05-22,1,False,False,False,False,None,None,None,525.0,LEO
2,Falcon 9,5eb0e4bbb6c3bb0006eeb1ed,CCSFS SLC 40,"{'core': '5e9e289ff3591884e03b262c', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",10,2013-03-01,1,False,False,False,False,None,None,None,677.0,ISS
3,Falcon 9,5eb0e4bbb6c3bb0006eeb1ee,VAFB SLC 4E,"{'core': '5e9e289ff359180ae23b262d', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': True, 'landing_success': False, 'landing_type': 'Ocean', 'landpad': None}",11,2013-09-29,1,False,False,False,True,False,Ocean,None,500.0,PO
4,Falcon 9,5eb0e4bbb6c3bb0006eeb1ef,CCSFS SLC 40,"{'core': '5e9e289ff35918862c3b262e', 'flight': 1, 'gridfins': False, 'legs': False, 'reused': False, 'landing_attempt': False, 'landing_success': None, 'landing_type': None, 'landpad': None}",12,2013-12-03,1,False,False,False,False,None,None,None,3170.0,GTO


In [480]:
tempp = temp_df.drop(['payloads','cores'],axis=1)
tempp['mass_kg'] = pd.to_numeric(tempp['mass_kg'],errors='coerce')
tempp.dropna(subset=['mass_kg'],inplace=True)
temp_df = tempp.copy()

In [493]:
spacex_df = temp_df.copy()

In [494]:
spacex_df.mass_kg.mean()

6123.547647058824